# Blocking: Attribute Equivalence Blocking

In [1]:
import sys
sys.path.insert(1, '../../')

from entityresolution.datasets import fetch_DBLP_ACM
from entityresolution.indexing.block.building import AttributeEquivalenceBlocking


## load dataset

In [2]:
golden_standard, amc, dblp  = fetch_DBLP_ACM()

  0%|          | 0.00/270k [00:00<?, ?B/s]

In [3]:
amc.head(3)

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999


In [4]:
dblp.head(3)

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
1,conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
2,conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002


In [5]:
amc = amc.set_index("id")
dblp = dblp.set_index("id")

ae = AttributeEquivalenceBlocking(left_on="title", right_on="title")
ae.build(amc, dblp).head()

title_l  \
index_l index_r                                                               
277955  conf/sigmod/Epstein95                               Keynote Address   
        conf/sigmod/Ellison95                               Keynote Address   
276377  conf/sigmod/OlstonWACELSS98                              DataSplash   
276367  conf/sigmod/Aulakh98               About Quark Digital Media System   
304211  journals/vldb/MaratheS02     Query processing techniques for arrays   

                                                                             authors_l  \
index_l index_r                                                                          
277955  conf/sigmod/Epstein95                                        Robert S. Epstein   
        conf/sigmod/Ellison95                                        Robert S. Epstein   
276377  conf/sigmod/OlstonWACELSS98  Chris Olston, Allison Woodruff, Alexander Aike...   
276367  conf/sigmod/Aulakh98                                              Kamar Aulakh   
304211  journals/vldb/MaratheS02                  Arunprasad P. Marathe, Kenneth Salem   

                                                                            venue_l  \
index_l index_r                                                                       
277955  conf/sigmod/Epstein95        International Conference on Management of Data   
        conf/sigmod/Ellison95        International Conference on Management of Data   
276377  conf/sigmod/OlstonWACELSS98  International Conference on Management of Data   
276367  conf/sigmod/Aulakh98         International Conference on Management of Data   
304211  journals/vldb/MaratheS02     International Conference on Management of Data   

                                     year_l  \
index_l index_r                               
277955  conf/sigmod/Epstein95          1995   
        conf/sigmod/Ellison95          1995   
276377  conf/sigmod/OlstonWACELSS98    1998   
276367  conf/sigmod/Aulakh98           1998   
304211  journals/vldb/MaratheS02       1999   

                                                                    title_r  \
index_l index_r                                                               
277955  conf/sigmod/Epstein95                               Keynote Address   
        conf/sigmod/Ellison95                               Keynote Address   
276377  conf/sigmod/OlstonWACELSS98                              DataSplash   
276367  conf/sigmod/Aulakh98               About Quark Digital Media System   
304211  journals/vldb/MaratheS02     Query processing techniques for arrays   

                                                                             authors_r  \
index_l index_r                                                                          
277955  conf/sigmod/Epstein95                                        Robert S. Epstein   
        conf/sigmod/Ellison95                                            Larry Ellison   
276377  conf/sigmod/OlstonWACELSS98  Vuk Ercegovac, Allison Woodruff, Michael Stone...   
276367  conf/sigmod/Aulakh98                                              Kamar Aulakh   
304211  journals/vldb/MaratheS02                  Arunprasad P. Marathe, Kenneth Salem   

                                               venue_r  year_r  \
index_l index_r                                                  
277955  conf/sigmod/Epstein95        SIGMOD Conference    1995   
        conf/sigmod/Ellison95        SIGMOD Conference    1995   
276377  conf/sigmod/OlstonWACELSS98  SIGMOD Conference    1998   
276367  conf/sigmod/Aulakh98         SIGMOD Conference    1998   
304211  journals/vldb/MaratheS02               VLDB J.    2002   

                                                                   AE_title  
index_l index_r                                                              
277955  conf/sigmod/Epstein95                               Keynote Address  
        conf/sigmod/Ellison95                               K

In [6]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Pre-sorted dataframes
def custom_merge(df1, df2, key):
    # Define the merging function
    def merge_records(i, j):
        return (i, j) if df1[key].loc[i] == df2[key].loc[j] else None

    # Using parallel processing to merge pre-sorted data
    merged_tuples = []

    with ThreadPoolExecutor() as executor:
        futures = []
        for i in range(len(df1)):
            for j in range(len(df2)):
                futures.append(executor.submit(merge_records, i, j))

        for future in futures:
            result = future.result()
            if result is not None:
                merged_tuples.append(result)

    print("Merged tuples of matched indexes:")
    return merged_tuples

In [10]:

custom_merge(amc, dblp, "title")


In [ ]:
%%timit
pd.merge(df_a, df_a, left_on="title", right_on="title", how='inner').index.tolist()